Building a Neural network using Convolutional Neural Networks and OpenCV to detect face emotions, with 7 target labels (angry,disgust,fear,happy,neutral,sad,surprise)

In [6]:
import os
import cv2

# Loading the Data

In [1]:
#Location of dataset

loc = "C:/Users/91808/Downloads/SPK IIIT/datasets/Emotion Detector Dataset/train/"

In [7]:
features = []
target = []
for i in ["angry","disgust","fear","happy","neutral","sad","surprise"]:
    
    collection_images_name = os.listdir(loc + str(i))
    for j in collection_images_name:
        img = cv2.imread(loc + str(i) + "/" + j)
        try:
            img = cv2.resize(img,(32,32))
        except:
            pass
        else:
            features.append(img)
            if i == "angry":
                target.append(0)
            elif i == "disgust":
                target.append(1)
            elif i == "fear":
                target.append(2)
            elif i == "happy":
                target.append(3)
            elif i == "neutral":
                target.append(4)
            elif i == "sad":
                target.append(5)
            elif i == "surprise":
                target.append(6)
    print("Loaded Folder ",i)     

Loaded Folder  angry
Loaded Folder  disgust
Loaded Folder  fear
Loaded Folder  happy
Loaded Folder  neutral
Loaded Folder  sad
Loaded Folder  surprise


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
#location of dataset

address = "C:/Users/91808/Downloads/SPK IIIT/datasets/Emotion Detector Dataset/"

In [10]:
dataSet_train = ImageDataGenerator()
dataSet_val = ImageDataGenerator()

train_features = dataSet_train.flow_from_directory(address + "train",
                                                   target_size=(48,48),
                                                   batch_size= 128,
                                                   color_mode="grayscale",
                                                   class_mode = "categorical",
                                                   )

test_features = dataSet_val.flow_from_directory(address + "validation",
                                                   target_size=(48,48),
                                                   batch_size= 128,
                                                   color_mode="grayscale",
                                                   class_mode = "categorical",
                                                
                                                   )

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


# Architecture

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,MaxPooling2D,Conv2D,Flatten,Dropout

In [12]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation="relu",input_shape = (48,48,1)))
model.add(Conv2D(64,(3,3),activation = "relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128,(3,3),activation="relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(7,activation="softmax"))

# Compilation

In [13]:
from tensorflow.keras.optimizers import Adam
model.compile(Adam(learning_rate=0.001),loss = "categorical_crossentropy",metrics = ["accuracy"])

# Training The Model

In [11]:

model.fit_generator(train_features,
                    steps_per_epoch= train_features.n // train_features.batch_size,
                    epochs= 50,
                    validation_data= test_features,
                    validation_steps= test_features.n // test_features.batch_size 
                    )

C:\Users\Nidhish\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
225/225 [==============================] - 168s 747ms/step - loss: 2.6726 - accuracy: 0.2469 - val_loss: 1.7562 - val_accuracy: 0.3048
Epoch 2/50
225/225 [==============================] - 123s 548ms/step - loss: 1.7427 - accuracy: 0.2910 - val_loss: 1.6939 - val_accuracy: 0.3328
Epoch 3/50
225/225 [==============================] - 123s 545ms/step - loss: 1.7021 - accuracy: 0.3127 - val_loss: 1.6408 - val_accuracy: 0.3602
Epoch 4/50
225/225 [==============================] - 136s 606ms/step - loss: 1.6600 - accuracy: 0.3347 - val_loss: 1.6074 - val_accuracy: 0.3778
Epoch 5/50
225/225 [==============================] - 136s 606ms/step - loss: 1.6103 - accuracy: 0.3601 - val_loss: 1.5786 - val_accuracy: 0.3814
Epoch 6/50
225/225 [==============================] - 136s 602ms/step - loss: 1.5221 - accuracy: 0.4053 - val_loss: 1.4626 - val_accuracy: 0.4347
Epoch 7/50
225/225 [==============================] - 137s 607ms/step - loss: 1.4572 - accuracy: 0.4291 - val_loss: 1.3805 -

# Saving the Model

In [ ]:
#Location of folder where, files of model are to saved

location = "C:/Users/91808/Downloads/SPK IIIT/internships and courses/python/emotion detector model files/"

In [12]:
ModelInJson = model.to_json()
abc = open(location + "model_emotion_detector.json","w")
abc.write(ModelInJson)

model.save_weights(location + "model_emotion_detector_weights.h5")

# Loading the Model

In [1]:
#Location of folder from where, files of model are to loaded

location = "C:/Users/91808/Downloads/SPK IIIT/internships and courses/python/emotion detector model files/"

In [2]:
from tensorflow.keras.models import model_from_json
abc = open(location + "model_emotion_detector.json","r")
loaded_data = abc.read()
loaded_model = model_from_json(loaded_data)
loaded_model.load_weights(location + "model_emotion_detector_weights.h5")

# Testing the Model

In [3]:
#location of haarcascade_frontalface_default file

place = "C:/Users/91808/Downloads/SPK IIIT/internships and courses/python/"

In [4]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

cv2.ocl.setUseOpenCL(False)

emotions = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}


cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
cap.set(10,180)

while True:

    ret, frame = cap.read()
    if not ret:
        break
    facecasc = cv2.CascadeClassifier(place + "haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = loaded_model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(frame,"Emotion: ",(20,35),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
        cv2.putText(frame,"Probablity: ",(20,75),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
        ProbabilityValue = np.amax(prediction)
        if ProbabilityValue> 0.25:
            cv2.putText(frame,emotions[maxindex],(225,35),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
            cv2.putText(frame,str(int(ProbabilityValue*100)) + "%",(225,75),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,225),2)
    cv2.imshow('Emotion Detector', cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
